In [45]:
import os

In [46]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml\\Loan-Amount-Prediction'

In [47]:
os.chdir("../")

In [48]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [49]:
project_path = "C:/Users/RICH-FILES/Desktop/ml/Loan-Amount-Prediction"

os.chdir(project_path)

In [50]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name:Path
    target_column: int
    mlflow_uri:str


In [51]:
from credit_risk.constants import *
from credit_risk.utils.common import read_yaml, create_directories, save_json

In [52]:
#creating a Configuration class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath   = CONFIG_FILE_PATH,
        params_filepath   = PARAMS_FILE_PATH,
        schema_filepath   = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)                       
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config= self.config.model_evaluation
        params= self.params.RandomForestRegressor
        schema= self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config=ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/richardmukechiwa/Loan-Amount-Prediction"
            
        )
        
        return model_evaluation_config
        

In [53]:
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import pandas as pd


In [54]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config
        
    def eval_metrics(self, actual, pred):
        rmse=np.sqrt(mean_squared_error(actual, pred))
        mae=mean_absolute_error(actual, pred)
        r2= r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)
        
        
        test_data.dropna(inplace=True)
        
        print(test_data.head())
       
        test_x=test_data.drop([self.config.target_column], axis=1)
        test_y= test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            
            predicted_qualities=model.predict(test_x)
            
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # saving metrics as local
            scores= {"rmse":rmse, "mae": mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            
            #Model registry does not work with file store
            #if tracking_url_type_store != "file":
                
                # Register the model
                
                # There are other ways to use the Model Registry, which depends on the use case
                
                # please refer to the docs: https://mlflow.org/docs/latest/models-reqistry.html # api-workflow
                # mlflow.sklearn.log_model(model, "model")
            #else:
                #mlflow.sklearn.log_model(model, "model")    
                
             
            
        
        
        

In [55]:
#define pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config=ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e    

[2025-03-22 00:33:33,640: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-22 00:33:33,643: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-22 00:33:33,649: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-22 00:33:33,651: INFO: common: created directory at: artifacts]
[2025-03-22 00:33:33,651: INFO: common: created directory at: artifacts/model_evaluation]
   Income      Home  Emp_length             Intent  Amount   Rate  \
0   22800      RENT         0.0            MEDICAL    6000  13.35   
1   43200      RENT         8.0            MEDICAL    7200  13.49   
2   85000  MORTGAGE         4.0            VENTURE    7000   6.91   
3   45000  MORTGAGE         0.0  DEBTCONSOLIDATION    7750  12.53   
4   33000      RENT         8.0            MEDICAL    6400   9.99   

   Percent_income  Cred_length  
0            0.26            4  
1            0.17            4  
2            0.08            3  
3            0.17            4 